# Pipeline: Gene-Drug association for CCLE cancer cell line pharmacogenomic data

Find significance of gene-drug association for an interested gene list with gene cnv, gene expression, drug response and dependency data, and box plot of drug/dep for cnv-status/gene-expression for the interested genes.

Updated August 22, 2019

fixed bug 08/21/2019: 
pearson correlation output 'nan' due to empty gene_cnv_drug for Gene_CNV_drug_TumorType(GeneID,drug,TumorType,cnv_cutoff)

@Copyright Zi-Ming Zhao ziming.gt@gmail.com

## Overall summary:
The pipeline is evaluating the drug effects of selected gene’s CNA changes by using the CCLE cell line genomic and drug data (CTRP v2) [Viswanathan et al. (Nature, 2017); Adams et al. (ACS Chem Biol, 2014)]. 

The Cancer Cell Line Encyclopedia (CCLE) cell line genomic and drug data (CTRP v2) were used [Viswanathan et al. (Nature, 2017); Adams et al. (ACS Chem Biol, 2014)]. The CCLE drug response data were downloaded from Cancer Therapeutics Response Portal (www.broadinstitute.org/ctrp), and CCLE gene-level CNA and gene expression data from depMap data portal (‘public_19Q1_gene_cn.csv’ and ‘CCLE_depMap_19Q1_TPM.csv’ https://depmap.org/portal/download/) [citations:
Cancer Cell Line Encyclopedia Consortium, and Genomics of Drug Sensitivity in Cancer Consortium. 2015. Pharmacogenomic Agreement between Two Cancer Cell Line Data Sets. Nature 528 (7580):84–87. https://doi.org/10.1038/nature15736.
Jordi Barretina, Giordano Caponigro, Nicolas Stransky, Kavitha Venkatesan, William R. Sellers, Robert Schlegel, Levi A. Garraway, et. al. 2012. The Cancer Cell Line Encyclopedia Enables Predictive Modelling of Anticancer Drug Sensitivity. Nature 483 (7391):603–7. https://doi.org/10.1038/nature11003.]. 

For CCLE drug data, area-under-concentration-response curve (AUC) sensitivity scores were used for each cancer cell line and each drug. In total, I collected gene-level CNA data from 668 CCLE cell lines, with a total of 545 drugs tested. With the CCLE gene-level CNA and AUC drug sensitivity scores, I performed drug effects functional analyses for selected  genes. Particularly, I took the selected gene list, calculated P-value from the Pearson correlation between each gene’s CNA and each drug across cell lines for the interested tumor types or pan-cancer, and further calculated Q-value by multiple testing Bonferroni correction for the P-values. Significant gene-CNA and drug associations were kept (Q-value > 0.1), and is further evaluated for the gene-expression and drug associations. Final significant genes were presented if the gene-drug association is significant for both CNA and gene expression in the same direction, e.g. drug sensitivity with both high CNA and high gene expression.

### Required input files/data:
CCLE_gene_shared_drug.csv #file with drug response auc for genes CNV for each cell line
CCLE_gene_shared_deps.csv file with dependency for gene CNV for each cell lines;
cnv_drug_sharedCellLines.csv ##file with IDs matching between drug/dependency cell lines and ccle cell lines
drug_target.csv

cnv_ctrpv2_gene_drug_tumortype_correlation.csv
gex_ctrpv2_gene_drug_tumortype_correlation.csv

### Output files:
Gene cnv/gene-expression and drug/dependency association.
Box plot of dep/drugs for cnv-status/gene-expression for the interested genes.

In [1]:
%matplotlib inline
import json
import pandas as pd
import numpy as np
import statistics as st
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy.stats.stats import pearsonr
from matplotlib_venn import venn3
from matplotlib_venn import venn2

In [110]:
### Check the genes' presence and absence in the datasets

genes=['GOLGA6L6','HLA-DQA1','HLA-DQB1','HLA-DRB1','HLA-DRB5','MACROD2','OR4M2','OR4N4','POTEB','POTEB2','RBFOX1','TPTE']

g1=ExistGenesWithoutCounts(genes,ccle_genes) 
g2=ExistGenesWithoutCounts(genes,ccle_gex_genes) 
g3=ExistGenesWithoutCounts(genes,ccle_cnv_genes) 
g4=ExistGenesWithoutCounts(genes,dep_ctrpv2_genes) 
g5=ExistGenesWithoutCounts(genes,drug_ctrpv2_genes) 

Missing Genes:
GOLGA6L6
POTEB
POTEB2
total_gene_number	12	exist_gene_number	9	Missing_gene_number	3
Missing Genes:
GOLGA6L6
HLA-DQA1
HLA-DQB1
HLA-DRB1
HLA-DRB5
MACROD2
OR4M2
OR4N4
POTEB
POTEB2
RBFOX1
TPTE
total_gene_number	12	exist_gene_number	0	Missing_gene_number	12
Missing Genes:
GOLGA6L6
HLA-DQA1
HLA-DQB1
HLA-DRB1
HLA-DRB5
MACROD2
OR4M2
OR4N4
POTEB
POTEB2
RBFOX1
TPTE
total_gene_number	12	exist_gene_number	0	Missing_gene_number	12
Missing Genes:
GOLGA6L6
MACROD2
OR4M2
OR4N4
POTEB
POTEB2
total_gene_number	12	exist_gene_number	6	Missing_gene_number	6
Missing Genes:
GOLGA6L6
MACROD2
OR4M2
OR4N4
POTEB
POTEB2
total_gene_number	12	exist_gene_number	6	Missing_gene_number	6


In [120]:
# get starting time

import time
start = time.time()
#genes=['GOLGA6L6','HLA-DQA1','HLA-DQB1','HLA-DRB1','HLA-DRB5','MACROD2','OR4M2','OR4N4','POTEB','POTEB2','RBFOX1','TPTE']
genes=['HLA-DQA1','HLA-DQB1','HLA-DRB1','HLA-DRB5','RBFOX1','TPTE']

output_f=SignificantGenes_cnv_drug(genes) # 5 min for 6 genes

#output_f=output_f[output_f['gex_r']!=0]
output_f.to_csv('Validated_Genes_DrugEffects_GeX.csv')

elapsed_time_fl = (time.time() - start) 
elapsed_time_fl

Number of genes tested: 
6
Nothing came out to be significant!


332.7235951423645

In [36]:
output_f.head()

,drug,gene_symbol_of_protein_target,tumor_type,Q_value,p_value,correlation_r,data_size,gex_dir,gex_r,gex_z,gex_num_cells,cnv_dir,cnv_r,cnv_z,cnv_num_cells,Q_value_dep,p_value_dep,correlation_r_dep,data_size_dep
gene,,,,,,,,,,,,,,,,,,,
RBFOX1,SKI-II,SPHK1,PanCancer,1.752631,0.000536,-0.167637,423.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.918196,0.319699,0.047013,450.0
TPTE,CAY10576,IKBKE,PanCancer,1.954234,0.000598,0.167614,416.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.932527,0.822088,-0.010629,450.0
RBFOX1,fluorouracil,TYMS,PanCancer,2.373052,0.000726,-0.160350,441.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.918196,0.319699,0.047013,450.0
RBFOX1,BMS-345541,IKBKB,PanCancer,8.132884,0.002487,-0.146541,424.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.918196,0.319699,0.047013,450.0
RBFOX1,BRD-K07442505,BAX,PanCancer,8.253115,0.002524,-0.143332,442.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.918196,0.319699,0.047013,450.0


In [27]:
d3=cnv_drug[cnv_drug['cpd_name']=='CIL55']
len(d3)

202

In [26]:
d2=cnv_drug[cnv_drug['cpd_name']=='MI-1']
len(d2)

393

In [25]:
d1=cnv_drug[cnv_drug['cpd_name']=='BRD-K09344309']
len(d1)

107

In [3]:
cnv_drug.head()

,area_under_curve,master_cpd_id.1,master_ccl_id,ccl_name,ccle_primary_site,cpd_name,gene_symbol_of_protein_target,ccl_name_mod.1,CCLE_Name.1
22RV1_PROSTATE,14.504,1788,7,22RV1,prostate,CIL55,NaN,22RV1_prostate,22RV1_PROSTATE
22RV1_PROSTATE,14.819,3588,7,22RV1,prostate,BRD4132,NaN,22RV1_prostate,22RV1_PROSTATE
22RV1_PROSTATE,14.101,12877,7,22RV1,prostate,BRD6340,NaN,22RV1_prostate,22RV1_PROSTATE
22RV1_PROSTATE,14.631,17712,7,22RV1,prostate,ML006,S1PR3,22RV1_prostate,22RV1_PROSTATE
22RV1_PROSTATE,13.506,18311,7,22RV1,prostate,Bax channel blocker,BAX,22RV1_prostate,22RV1_PROSTATE


In [7]:
cnv.head()

,CCLE_Name.1,Aliases,COSMIC_ID,Sanger ID,Primary Disease,Subtype Disease,Gender,Source,A1BG,NAT2,...,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,KCNE2,DGCR2,CASP8AP2,SCO2
CCLE_Name,,,,,,,,,,,,,,,,,,,,,
NIHOVCAR3_OVARY,NIHOVCAR3_OVARY,NIH:OVCAR-3;OVCAR3,905933.0,2201.0,Ovarian Cancer,"Adenocarcinoma, high grade serous",Female,ATCC,0.261700,0.105800,...,-0.145900,-0.008000,-0.213800,0.705100,-0.612700,-0.09390,0.671700,0.453900,-0.036500,-0.57280
HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HL-60,905938.0,55.0,Leukemia,"Acute Myelogenous Leukemia (AML), M3 (Promyelo...",Female,ATCC,-0.006700,0.069000,...,0.059600,0.025900,0.051200,0.604500,0.066400,0.08950,0.014800,0.135600,0.023100,0.07230
CACO2_LARGE_INTESTINE,CACO2_LARGE_INTESTINE,CACO2;CACO2;CaCo-2,NaN,NaN,Colon/Colorectal Cancer,Colon Adenocarcinoma,-1,NaN,0.045300,0.132000,...,0.708000,-0.157000,0.223700,-0.011700,0.331900,-0.25770,-0.028300,-0.002000,-0.060700,-0.09420
HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HEL,907053.0,783.0,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",Male,DSMZ,0.432200,0.119700,...,0.140500,-0.382300,-0.355400,0.064700,-0.375800,0.16930,0.988900,0.190500,0.166800,-0.38020
HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HEL 92.1.7,NaN,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",Male,ATCC,0.265953,0.215723,...,0.194863,-0.280401,-0.382478,-0.327153,-0.399479,0.21851,1.165456,-0.340663,0.190381,-0.39753


In [2]:
# Open cell line gene cnv file:
cnv_f = '/Users/zhaoz/Dropbox/gene_drug_pipeline/CCLE_gene_shared_drug.csv'
cnv = pd.read_csv(cnv_f,sep=',',index_col=0)
cnv.head() #(668, 16341)
# File 'cnv_drug_sharedCellLines.csv' with compound, target, AUC, and shared cell lines with cnv files.
file_shared='/Users/zhaoz/Dropbox/gene_drug_pipeline/cnv_drug_sharedCellLines.csv'
cnv_drug= pd.read_csv(file_shared,sep=',',index_col=0)
cnv_drug.head() #545 drugs
dep_f = '/Users/zhaoz/Dropbox/gene_drug_pipeline/CCLE_gene_shared_deps.csv'
dep = pd.read_csv(dep_f,sep=',',index_col=0)
dep.head() #16341 columns
cell_line=pd.concat([cnv_drug['CCLE_Name.1'],cnv_drug['ccle_primary_site']],axis=1)
cell_lines=cell_line.drop_duplicates()
dep_mod=pd.concat([cell_lines,dep],axis=1,join='inner')
dep_mod.head()

# genes in dependency and drug data
dep_ctrpv2_genes=dep_mod.T[10:]
dep_ctrpv2_genes=dep_ctrpv2_genes.index
dep_ctrpv2_genes
drug_ctrpv2_genes=cnv.T[8:]
drug_ctrpv2_genes=drug_ctrpv2_genes.index
drug_ctrpv2_genes

# Open file drug targeted gene "drug_target.csv"
file_drug_target='/Users/zhaoz/Dropbox/gene_drug_pipeline/drug_target.csv'
drug_target= pd.read_csv(file_drug_target,sep=',',index_col=0)
drug_target.index=drug_target['cpd_name']
drug_target.head()

### CCLE validation data: gene expression and cnv vs drug
# Open cell line gene cnv file:
f_cnv = '/Users/zhaoz/Dropbox/gene_drug_pipeline/cnv_ctrpv2_gene_drug_tumortype_correlation.csv'
cnv_corr = pd.read_csv(f_cnv,sep=',',index_col=0)
cnv_corr['context']=cnv_corr['context'].str.replace('all_CCLs','PanCancer')
cnv_corr.head() #(195214, 8)
# Open cell line gene expression file:
f_gex = '/Users/zhaoz/Dropbox/gene_drug_pipeline/gex_ctrpv2_gene_drug_tumortype_correlation.csv'
gex = pd.read_csv(f_gex,sep=',',index_col=0)
gex['context']=gex['context'].str.replace('all_CCLs','PanCancer')

gex.head() #(1184225, 8)
# genes found in the ccle gex and cnv data
ccle_gex_genes=gex['hugo_gene_symbol']
ccle_cnv_genes=cnv_corr['hugo_gene_symbol']
ccle_genes=ccle_gex_genes.append(ccle_cnv_genes)
ccle_genes=ccle_genes.unique() #16989

drugs=cnv_drug['cpd_name'].unique() #545 drugs including dosing difference
num_drugs=len(drugs)
interested_tumor_types=['PanCancer']
num_tumorTypes=len(interested_tumor_types)
num_tumorTypes #1
num_drugs #545

/Users/zhaoz/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


545

In [109]:
# cnv-drug q value cutoff
def SignificantGenes_cnv_drug(gene_list):
    test_genes=gene_list
    test_drugs=drugs
    num_genes=len(test_genes)
    TotalTests=num_genes*num_drugs*num_tumorTypes
    TotalTests_dep=num_genes*num_tumorTypes
    
    Gene_drug_tumor=pd.DataFrame()
    df=pd.DataFrame()
    cnv_cutoff=0.2
    p_value_cutoff=0.05
    q_value_cutoff=0.1
    min_data=5;
    min_cutoff=min_data-1;
    gene_names=''
    i=0
    count=0
    print ("Number of genes tested: ")
    print (num_genes)
    for gene in test_genes:
        i=i+1
        gene_names=gene+'_'+gene_names
        order=str(i)+':'+gene
        #print (order)
        for tumor in interested_tumor_types: 
            (tmp_r_dep,tmp_p_dep,data_size_dep)=Gene_CNV_dep_TumorType(gene,tumor)
            tmp_q_dep=tmp_p_dep*TotalTests_dep       
            for drug in test_drugs:
                (tmp_r,tmp_p,data_size)=Gene_CNV_drug_TumorType(gene,drug,tumor,cnv_cutoff)           
                tmp_q=tmp_p*TotalTests
                if (tmp_q<q_value_cutoff):
                    count=count+1
                    #gene_count=ExsitGenes_WithCounts[gene]                    
                if (tumor=='PanCancer'):
                    validate_tumor='all_CCLs'
                else:
                    validate_tumor=tumor

                (cnv_direction,cnv_peason_corr, cnv_zscore,cnv_num_cells)=find_gene_gex(gene,validate_tumor,drug,cnv_corr)
                (gex_direction,gex_peason_corr, gex_zscore,gex_num_cells)=find_gene_gex(gene,validate_tumor,drug,gex)
                df = df.append({'gene':gene, 'drug':drug,'tumor_type':tumor,'correlation_r':tmp_r,'p_value':tmp_p,'Q_value':tmp_q,'data_size':data_size,'gex_dir':gex_direction,'gex_r':gex_peason_corr, 'gex_z':gex_zscore,'gex_num_cells':gex_num_cells,'cnv_dir':cnv_direction,'cnv_r':cnv_peason_corr, 'cnv_z':cnv_zscore,'cnv_num_cells':cnv_num_cells,'correlation_r_dep':tmp_r_dep,'p_value_dep':tmp_p_dep,'Q_value_dep':tmp_q_dep,'data_size_dep':data_size_dep}, ignore_index=True)

    df.index=df['drug']
    output_content=pd.concat([df,drug_target],axis=1,join='inner') 
    output_content.index=output_content['gene']

    # re-organize the columns
    dc=output_content
    Gene_drug_tumor=pd.concat([dc['drug'],dc['gene_symbol_of_protein_target'],dc['tumor_type'],dc['Q_value'],dc['p_value'],dc['correlation_r'],dc['data_size'],dc['gex_dir'],dc['gex_r'],dc['gex_z'],dc['gex_num_cells'],dc['cnv_dir'],dc['cnv_r'],dc['cnv_z'],dc['cnv_num_cells'],dc['Q_value_dep'],dc['p_value_dep'],dc['correlation_r_dep'],dc['data_size_dep']],axis=1,join='inner')
    Gene_drug_tumor.index=dc['gene']
    # rank by p_value
    Gene_drug_tumor.sort_values("Q_value", inplace = True) 
    size=Gene_drug_tumor.shape[0]
        
    if (count>0): # save the file if it is not empty          
        return (Gene_drug_tumor)

    else:
        print ("Nothing came out to be significant!")
        return (Gene_drug_tumor)

In [119]:
### Main function to take the gene name, drug name, tumor time, and cnv_cutoff as input files;
#### and output the box plot and pearson correlation r and p values
### For pan-cancer, use TumorType as All
### fixed bug 08/21/2019: pearson correlation output 'nan' due to empty gene_cnv_drug.

def Gene_CNV_drug_TumorType(GeneID,drug,TumorType,cnv_cutoff):
    import seaborn as sns
    import matplotlib.pyplot as plt
    if (TumorType == "PanCancer"):
        drug_tumor=cnv_drug
    else:
        drug_tumor=cnv_drug[cnv_drug['ccle_primary_site']==TumorType]    
    gene_cnv=cnv[GeneID]
    gene_drug=drug_tumor[drug_tumor['cpd_name']==drug]    
    
    # fix bug: two few drug data or tumor type data, skip it. size of gene_cnv_drug
    # Save data size
    gene_cnv_drug=pd.concat([gene_drug,gene_cnv],axis=1,join='inner') 
    rows=gene_cnv_drug.shape[0]
    #print (gene_cnv_drug.head())

    if rows<3:
        print (GeneID+" "+drug+" "+TumorType+" Not enough data!")
        return (0,1,int(rows)) # if not enough data, force r=0 and p=1
    else:
        
        gene_cnv_drug.columns.values[-1] = "cnv"
        gene_cnv_drug.columns.values[0] = "AUC"
        
        gene_cnv_drug=gene_cnv_drug.sort_values(by='cnv', ascending=True)
        #print (gene_cnv_drug.head())

        (r,p)=pearsonr(gene_cnv_drug['cnv'],gene_cnv_drug['AUC'])
        #print (r,p)
        return (r,p,int(rows))

In [108]:
Gene_CNV_drug_TumorType('HLA-DRB1','CIL55','PanCancer',0.2)

0.051827038124584855 0.4638500901233997


(0.051827038124584855, 0.4638500901233997, 202)

In [92]:
Gene_CNV_drug_TumorType_correlation('HLA-DRB1','CIL55','PanCancer',0.2)

0.051827038124584855 0.4638500901233997


(0.051827038124584855, 0.4638500901233997, 202)

In [121]:
### Test plotting function to take the gene name, drug name, tumor time, and cnv_cutoff as input files;
#### and output the box plot and pearson correlation r and p values
### For pan-cancer, use TumorType as All

def Gene_CNV_drug_TumorType_correlation(GeneID,drug,TumorType,cnv_cutoff):
    import seaborn as sns
    import matplotlib.pyplot as plt
    if (TumorType == "PanCancer"):
        drug_tumor=cnv_drug
    else:
        drug_tumor=cnv_drug[cnv_drug['ccle_primary_site']==TumorType]    
    gene_cnv=cnv[GeneID]
    gene_drug=drug_tumor[drug_tumor['cpd_name']==drug]    
    
    # fix bug: two few drug data or tumor type data, skip it. size of gene_cnv_drug
    # Save data size
    gene_cnv_drug=pd.concat([gene_drug,gene_cnv],axis=1,join='inner') 
    rows=gene_cnv_drug.shape[0]
    #print (gene_cnv_drug.head())

    if rows<3:
        print (GeneID+" "+drug+" "+TumorType+" Not enough data!")
        return (0,1,int(rows)) # if not enough data, force r=0 and p=1
    else:
        
        gene_cnv_drug.columns.values[-1] = "cnv"
        gene_cnv_drug.columns.values[0] = "AUC"
        
        gene_cnv_drug=gene_cnv_drug.sort_values(by='cnv', ascending=True)
        #print (gene_cnv_drug.head())

        (r,p)=pearsonr(gene_cnv_drug['cnv'],gene_cnv_drug['AUC'])
        #print (r,p)
        return (r,p,int(rows))

In [115]:
### Create a function to take the gene name as the input and output the box plot

def Gene_CNV_dep_TumorType(GeneID,TumorType):
    import seaborn as sns
    import matplotlib.pyplot as plt
    if (TumorType == "PanCancer"):
        dep_tumor=dep_mod
    else:
        dep_tumor=dep_mod[dep_mod['ccle_primary_site']==TumorType]    
    gene_cnv=cnv[GeneID]
    gene_cnv=gene_cnv.rename(columns={GeneID:'cnv'}, inplace=True)
    gene_dep=dep_tumor[GeneID]
    gene_cnv_dep=pd.concat([gene_cnv,gene_dep],axis=1,join='inner') 
    gene_cnv_dep.columns.values[0] = "cnv"
    gene_cnv_dep.columns.values[1] = "dep"
    gene_cnv_dep=gene_cnv_dep.sort_values(by='cnv', ascending=True)
    gene_cnv_dep=gene_cnv_dep.dropna()
    (r,p)=pearsonr(gene_cnv_dep['cnv'],gene_cnv_dep['dep'])
    cellLineNum=gene_cnv_dep.shape[0]
    return (r,p,int(cellLineNum))   


In [31]:
### Function to validate the gene, drug, tumor_type and data 
def find_gene_gex(gene,tumor_type,drug,data):
    g=gene
    t=tumor_type
    d=drug
    f=data
    f_t=f[f['context']==t]
    f_t_g=f_t[f_t['hugo_gene_symbol']==g]
    indx=f_t_g.index
    if (indx.contains(d)):
        tmp=f_t_g.loc[d,:]
        
        return (tmp['correl_direction'],tmp['pearson_correlation'],tmp['correlation_zscore'],tmp['number_of_cell_lines'])
    else:
        return (0,0,0,0)

In [32]:
### Function to validate the gene, drug,and data for a fixed tumor type
def find_gene_drug(gene,drug,data):
    g=gene
    d=drug
    f=data
    indx=f.index
    if (indx.contains(d)):
        tmp=f.loc[d,:]        
        return (tmp['correl_direction'],tmp['pearson_correlation'],tmp['correlation_zscore'],tmp['number_of_cell_lines'])
    else:
        return (0,0,0,0)

In [33]:
# Function to get exist genes from a query gene list [e.g. co-exisiting ccle genes]
# input: test_gene_with_counts (dtype: series), query_gene_list
# output: exist_gene_with_counts
# Example: ExistGenesWithCounts(PTP0_all,ccle_genes); ExistGenesWithCounts(PTP0_all,cnv)
def ExistGenesWithoutCounts(test_gene_without_counts,query_gene_list):
    test_genes=test_gene_without_counts
    #print (test_genes)
    Exist_genes=[]
    NotExist=[]
    print ("Missing Genes:")
    for gene in test_genes:
        #print (gene)
        if gene in query_gene_list:
            Exist_genes.append(gene)
        else:
            NotExist.append(gene)
            print (gene)
    p1='total_gene_number\t'+str(len(test_genes))+"\texist_gene_number\t"+str(len(Exist_genes))+"\tMissing_gene_number\t"+str(len(NotExist))
    print (p1)
    #print (NotExist)

    #rowData = dfObj.loc[ ['c' , 'b'] , : ]
    return (Exist_genes)

In [34]:
### Create a function to take the gene name as the input and output the box plot
# Separate the data by GeneID1, the status of CNV, and look at gene1's CNV status's impact on gene2's genetic dependency

def Gene_CNV_dep_plotting(GeneID1_CNV,GeneID2_dep,TumorType,cnv_cutoff):
    import seaborn as sns
    import matplotlib.pyplot as plt
    if (TumorType == "PanCancer"):
        dep_tumor=dep_mod
    else:
        dep_tumor=dep_mod[dep_mod['ccle_primary_site']==TumorType]    
    gene_cnv=cnv[GeneID1_CNV]
    x_axis=GeneID1_CNV+'_cnv'
    gene_cnv=gene_cnv.rename(columns={GeneID1_CNV:x_axis}, inplace=True)
    gene_dep=dep_tumor[GeneID2_dep]
    gene_cnv_dep=pd.concat([gene_cnv,gene_dep],axis=1,join='inner') 
    
    y_axis=GeneID2_dep+'_genetic_dependency'
    gene_cnv_dep.columns.values[0] = x_axis
    gene_cnv_dep.columns.values[1] = 'genetic_dependency'
    gene_cnv_dep=gene_cnv_dep.dropna()
    (r,p)=pearsonr(gene_cnv_dep[x_axis],gene_cnv_dep['genetic_dependency'])
    cellLineNum=gene_cnv_dep.shape[0]

    
    gene_cnv_dep=gene_cnv_dep.sort_values(by=x_axis, ascending=True)
    gene_cnv_s=gene_cnv_dep[x_axis]
    gain_cnv = gene_cnv_s > cnv_cutoff; #default 0.2
    gain_cnv
    gain_cnv.sum() #10 pts
    gene_cnv_dep_sort=pd.concat([gene_cnv_dep,gain_cnv],axis=1,join='inner') 
    

    gene_cnv_dep_sort.columns.values[2] = 'cnv_status'
    gene_cnv_dep_sort.head()
    gene_cnv_dep_sort=gene_cnv_dep_sort.dropna()
    x=gene_cnv_dep_sort['cnv_status']
    y=gene_cnv_dep_sort['genetic_dependency']
    order=['NoGain','Gain']
    df=gene_cnv_dep_sort
    ax = sns.boxplot(x=x,y=y)
    
    TumorType=TumorType.replace('/','')
    TumorType=TumorType.replace(' ','')
    fig, ax = plt.subplots(1, 1)
    plt.xlim(-2,2)
    ax.plot(gene_cnv_dep[x_axis],gene_cnv_dep['genetic_dependency'], '.')
    ax.set_xlabel(x_axis)
    ax.set_ylabel(y_axis)

    return (r,p,int(cellLineNum))  

In [71]:
### Test plotting function to take the gene name, drug name, tumor time, and cnv_cutoff as input files;
#### and output the box plot and pearson correlation r and p values
### For pan-cancer, use TumorType as All

def Gene_CNV_drug_TumorType_plotting(GeneID,drug,TumorType,cnv_cutoff):
    import seaborn as sns
    import matplotlib.pyplot as plt
    if (TumorType == "PanCancer"):
        drug_tumor=cnv_drug
    else:
        drug_tumor=cnv_drug[cnv_drug['ccle_primary_site']==TumorType]    
    gene_cnv=cnv[GeneID]
    #gene_cnv.columns=['cnv']
    #gene_cnv=gene_cnv.rename(columns={GeneID:'cnv'}, inplace=True)
    #print (gene_cnv.head())
    gene_drug=drug_tumor[drug_tumor['cpd_name']==drug]    
    
    # fix bug: two few drug data or tumor type data, skip it. size of gene_cnv_drug
    # Save data size
    gene_cnv_drug=pd.concat([gene_drug,gene_cnv],axis=1,join='inner') 
    rows=gene_cnv_drug.shape[0]
    #print (gene_cnv_drug.head())

    if rows<3:
        print (GeneID+" "+drug+" "+TumorType+" Not enough data!")
        return (0,1,int(rows)) # if not enough data, force r=0 and p=1
    else:
        
        gene_cnv_drug.columns.values[-1] = "cnv"
        gene_cnv_drug.columns.values[0] = "AUC"
        print (gene_cnv_drug.head())
        gene_cnv_drug=gene_cnv_drug.sort_values(by='cnv', ascending=True)
        gene_cnv_s=gene_cnv_drug['cnv']
        gain_cnv = gene_cnv_s > cnv_cutoff; #default 0.2
        gain_cnv
        gain_cnv.sum() #10 pts
        gene_cnv_drug_sort=pd.concat([gene_cnv_drug,gain_cnv],axis=1,join='inner') 
        gene_cnv_drug_sort.columns.values[-1] = 'CNA_status'
        gene_cnv_drug=gene_cnv_drug.dropna()
        
        
        x=gene_cnv_drug['cnv']
        y=gene_cnv_drug['AUC']
        newdf=pd.concat([gene_cnv_drug['cnv'], gene_cnv_drug['AUC']], axis=1, join='inner', sort=False)
        newdf.dropna()
        print (gene_cnv_drug.head())
        #print (x)
        #print (y)
        #nas = np.logical_or(x.isnan(), y.isnan())
        #(r,p)=pearsonr(x[~nas], y[~nas])
        (r,p)=pearsonr(newdf['cnv'],newdf['AUC'])
        print (r,p)
        x_label=GeneID+'_CNA_status'
        y_label=drug+'_AUC'
        ax = sns.boxplot(x=gene_cnv_drug_sort['CNA_status'], y=gene_cnv_drug_sort['AUC'])
        order=['NoGain','Gain']
        TumorType=TumorType.replace('/','')
        TumorType=TumorType.replace(' ','')


        fig, ax = plt.subplots(1, 1)
        plt.xlim(-2,2)
        ax.plot(gene_cnv_drug['cnv'],gene_cnv_drug['AUC'], '.')
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        
        file_name=GeneID+"_"+drug+"_"+TumorType+'_cna_cutoff_'+str(cnv_cutoff)+'_gain_cna_drug.csv'
        #gene_cnv_drug_sort['CNA_status']=gene_cnv_drug_sort['CNA_status'].apply(ChangeTrueFalse)
        gene_cnv_drug_sort.to_csv(file_name)
        
        return (r,p,int(rows))

    # Add gene name and p-value to the box plot

In [78]:
### Test plotting function to take the gene name, drug name, tumor time, and cnv_cutoff as input files;
#### and output the box plot and pearson correlation r and p values
### For pan-cancer, use TumorType as All

def Gene_CNV_drug_TumorType_correlation_plotting(GeneID,drug,TumorType,cnv_cutoff):
    import seaborn as sns
    import matplotlib.pyplot as plt
    if (TumorType == "PanCancer"):
        drug_tumor=cnv_drug
    else:
        drug_tumor=cnv_drug[cnv_drug['ccle_primary_site']==TumorType]    
    gene_cnv=cnv[GeneID]
    #gene_cnv.columns=['cnv']
    #gene_cnv=gene_cnv.rename(columns={GeneID:'cnv'}, inplace=True)
    #print (gene_cnv.head())
    gene_drug=drug_tumor[drug_tumor['cpd_name']==drug]    
    
    # fix bug: two few drug data or tumor type data, skip it. size of gene_cnv_drug
    # Save data size
    gene_cnv_drug=pd.concat([gene_drug,gene_cnv],axis=1,join='inner') 
    rows=gene_cnv_drug.shape[0]
    #print (gene_cnv_drug.head())

    if rows<3:
        print (GeneID+" "+drug+" "+TumorType+" Not enough data!")
        return (0,1,int(rows)) # if not enough data, force r=0 and p=1
    else:
        
        gene_cnv_drug.columns.values[-1] = "cnv"
        gene_cnv_drug.columns.values[0] = "AUC"
        
        gene_cnv_drug=gene_cnv_drug.sort_values(by='cnv', ascending=True)
        print (gene_cnv_drug.head())

        (r,p)=pearsonr(gene_cnv_drug['cnv'],gene_cnv_drug['AUC'])
        print (r,p)
        x_label=GeneID+'_CNA_status'
        y_label=drug+'_AUC'
        TumorType=TumorType.replace('/','')
        TumorType=TumorType.replace(' ','')


        fig, ax = plt.subplots(1, 1)
        plt.xlim(-2,2)
        ax.plot(gene_cnv_drug['cnv'],gene_cnv_drug['AUC'], '.')
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        
        file_name=GeneID+"_"+drug+"_"+TumorType+'_cna_cutoff_'+str(cnv_cutoff)+'_gain_cna_drug.csv'
        #gene_cnv_drug_sort['CNA_status']=gene_cnv_drug_sort['CNA_status'].apply(ChangeTrueFalse)
        gene_cnv_drug.to_csv(file_name)
        
        return (r,p,int(rows))

    # Add gene name and p-value to the box plot